## Version 1.1

In [1]:
from sentence_transformers import SentenceTransformer, util

# Memuat model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Mendefinisikan dua kalimat
kalimat1 = "Ini adalah contoh kalimat pertama."
kalimat2 = "Ini adalah contoh kalimat kedua."

# Menghitung embeddings untuk kedua kalimat
embeddings1 = model.encode(kalimat1, convert_to_tensor=True)
embeddings2 = model.encode(kalimat2, convert_to_tensor=True)

# Menghitung kesamaan kosinus antara dua embeddings
kesamaan = util.pytorch_cos_sim(embeddings1, embeddings2)

print(f"Skor Kesamaan: {kesamaan.item()}")

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Skor Kesamaan: 0.8118517994880676


In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Memuat model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Dataframe
data_lite = {'Nama': ['Andi', 'Budi', 'Anton'],
             'Jawaban': ['Revolusi Industri 4.0 merupakan periode transformasi industri yang ditandai oleh integrasi teknologi digital ke dalam proses produksi.',
                         'Revolusi Industri 4.0 merupakan periode transformasi industri yang ditandai oleh integrasi teknologi digital ke dalam proses produksi.',
                         'Revolusi Industri 4.0 merupakan upaya transformasi menuju perbaikan dengan mengintegrasikan dunia online dan lini produksi di industri']}
df = pd.DataFrame(data_lite)

# Mendefinisikan kalimat
kalimat = df['Jawaban']

# Menghitung embeddings untuk kalimat
embeddings = model.encode(kalimat.tolist(), convert_to_tensor=True)

# Menghitung kesamaan kosinus antara setiap pasangan kalimat
for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        kesamaan = util.pytorch_cos_sim(embeddings[i], embeddings[j])
        print(f"Skor Kesamaan antara kalimat {i+1} dan {j+1}: {kesamaan.item()}")


Skor Kesamaan antara kalimat 1 dan 2: 1.0000001192092896
Skor Kesamaan antara kalimat 1 dan 3: 0.8827536702156067
Skor Kesamaan antara kalimat 2 dan 3: 0.8827536702156067


## Version 1.2

In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Memuat model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Dataframe
data_lite = {'Nama': ['Andi', 'Budi', 'Anton'],
             'Jawaban': ['Revolusi Industri 4.0 merupakan periode transformasi industri yang ditandai oleh integrasi teknologi digital ke dalam proses produksi.',
                         'Revolusi Industri 4.0 merupakan periode transformasi industri yang ditandai oleh integrasi teknologi digital ke dalam proses produksi.',
                         'Revolusi Industri 4.0 merupakan upaya transformasi menuju perbaikan dengan mengintegrasikan dunia online dan lini produksi di industri']}
df = pd.DataFrame(data_lite)

# Mendefinisikan kalimat
kalimat = df['Jawaban']

# Menghitung embeddings untuk kalimat
embeddings = model.encode(kalimat.tolist(), convert_to_tensor=True)

# Inisialisasi matriks similarity dengan nol
num_sentences = len(kalimat)
similarity_matrix = np.zeros((num_sentences, num_sentences))

# Mengisi matriks similarity
for i in range(num_sentences):
    for j in range(i + 1, num_sentences):
        similarity_matrix[i][j] = util.pytorch_cos_sim(embeddings[i], embeddings[j])

# Mengisi matriks dengan simetri diagonal
for i in range(num_sentences):
    for j in range(0, i):
        similarity_matrix[i][j] = similarity_matrix[j][i]

# Membuat dataframe dari matriks similarity
similarity_df = pd.DataFrame(similarity_matrix, columns=df['Nama'], index=df['Nama'])

print("Similarity Matrix:")
print(similarity_df)

Similarity Matrix:
Nama       Andi      Budi     Anton
Nama                               
Andi   0.000000  1.000000  0.882754
Budi   1.000000  0.000000  0.882754
Anton  0.882754  0.882754  0.000000


## Version 1.3

In [ ]:
import pandas as pd
 
def preprocess_data(data_path, sample_size):
 
  # Read the data from specific path
  data = pd.read_csv(data_path, low_memory=False)

  # Drop articles without Abstract
  data = data.dropna(subset = ['abstract']).reset_index(drop = True)

  # Get "sample_size" random articles
  data = data.sample(sample_size)[['abstract']]
 
 return data
 
# Read data & preprocess it
data_path = "./data/cord19_source_data.csv"

In [ ]:
# Useful libraries
import numpy as np
import torch
from keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer,  AutoModelForSequenceClassification
 
# Load bert model
model_path = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_path,
                                         do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                         output_attentions=False,
                                                         output_hidden_states=True)
                                                        
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
 
def create_vector_from_text(tokenizer, model, text, MAX_LEN = 510):
  
    input_ids = tokenizer.encode(
                        text,
                        add_special_tokens = True,
                        max_length = MAX_LEN,                          
    )   
    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long",
                              truncating="post", padding="post")
    # Remove the outer list.
    input_ids = results[0]
    # Create attention masks   
    attention_mask = [int(i>0) for i in input_ids]
    # Convert to tensors.
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    # Add an extra dimension for the "batch" (even though there is only one
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers.
    with torch.no_grad():       
        logits, encoded_layers = model(
                                    input_ids = input_ids,
                                    token_type_ids = None,
                                    attention_mask = attention_mask,
                                    return_dict=False)

    layer_i = 12 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]
      
    # Extract the vector.
    vector = encoded_layers[layer_i][batch_i][token_i]
    # Move to the CPU and convert to numpy ndarray.
    vector = vector.detach().cpu().numpy()
    return(vector)
 
def create_vector_index(data):
  
   # The list of all the vectors
   vectors = []
  
   # Get overall text data
   source_data = data.abstract.values
  
   # Loop over all the comment and get the embeddings
   for text in tqdm(source_data):
      
       # Get the embedding
       vector = create_vector_from_text(tokenizer, model, text)
      
       #add it to the list
       vectors.append(vector)
  
   data["vectors"] = vectors
   data["vectors"] = data["vectors"].apply(lambda emb: np.array(emb))
   data["vectors"] = data["vectors"].apply(lambda emb: emb.reshape(1, -1))
   return data
# Create the vector index
vector_index = create_vector_index(source_data)
vector_index.sample(5)